<a href="https://www.kaggle.com/code/gargivipat/poem-generator?scriptVersionId=164731623" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/poems/poems-100.csv


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical

2024-02-28 16:55:11.881482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 16:55:11.881595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 16:55:12.056394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/poems/poems-100.csv")

In [4]:
df.head()

,text
0,"O my Luve's like a red, red rose\nThat’s newly..."
1,"The rose is red,\nThe violet's blue,\nSugar is..."
2,How do I love thee? Let me count the ways.\nI ...
3,"Had I the heavens' embroidered cloths,\nEnwrou..."
4,"I.\n Enough! we're tired, my heart and I.\n..."


In [5]:
df.shape

(100, 1)

In [6]:
df['text'][0]

"O my Luve's like a red, red rose\nThat’s newly sprung in June;\nO my Luve's like the melodie\nThat’s sweetly play'd in tune.\n\nAs fair art thou, my bonnie lass,\nSo deep in luve am I:\nAnd I will luve thee still, my dear,\nTill a’ the seas gang dry:\n\nTill a’ the seas gang dry, my dear,\nAnd the rocks melt wi’ the sun:\nI will luve thee still, my dear,\nWhile the sands o’ life shall run.\n\nAnd fare thee well, my only Luve\nAnd fare thee well, a while!\nAnd I will come again, my Luve,\nTho’ it were ten thousand mile."

In [7]:
len(df['text'][0])

506

In [8]:
text = df['text'].values

In [9]:
len(text)

100

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
tokens = tokenizer.texts_to_sequences(text)

In [11]:
# tokenizer.get_config()

In [12]:
X = []
y = []
for t in tokens:
    for i in range(1,len(t)):
        X.append(t[:i])
        y.append(t[i])

In [13]:
# help(tokenizer)

In [14]:
# len(tokenizer.index_word)

In [15]:
vocab_size = len(tokenizer.index_word)+1

In [16]:
X = pad_sequences(X,padding="pre")
y = to_categorical(y,num_classes=vocab_size)

In [17]:
y[100]

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
ind = 0
for i,num in enumerate(y[100]):
    if num==1:
        ind = i

In [19]:
ind

13

In [20]:
tokenizer.index_word[13]

'it'

In [21]:
len(X[100])

2026

In [22]:
X

array([[   0,    0,    0, ...,    0,    0,  105],
       [   0,    0,    0, ...,    0,  105,    8],
       [   0,    0,    0, ...,  105,    8, 1175],
       ...,
       [   0,    0,    0, ...,   26,    1,  233],
       [   0,    0,    0, ...,    1,  233,    3],
       [   0,    0,    0, ...,  233,    3, 5396]], dtype=int32)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [24]:
embed_dim = 300
neurons = 256

In [25]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [26]:
def sparse_cat_model(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import *
from keras.losses import * 

In [28]:
from tensorflow.keras import backend as back
in_shape = Input(shape=(None,))
a = Embedding(vocab_size,embed_dim)(in_shape)
b = GRU(neurons,return_sequences=True)(a)
b = GRU(neurons,return_sequences=True)(b)
c = Dense(1,activation='tanh')(b)
r = Reshape([-1])(c)
alpha = Activation('softmax')(r)
e = Permute([2,1])(RepeatVector(neurons)(alpha))
e = Multiply()([b,e])
e = Lambda(lambda xin : back.sum(xin,axis=1),output_shape=(neurons,))(e)
out = Dense(vocab_size,activation='softmax',name='output')(e)
# out = Reshape((-1,vocab_size))(out)

# target = Input(shape=(None,))
# out = Lambda(lambda x: back.expand_dims(x,axis=-1))(target)

In [29]:
model = tf.keras.models.Model(inputs=[in_shape],outputs=[out])
model.compile(optimizer='adam',loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

In [31]:
early_stop = EarlyStopping(monitor='accuracy',patience=2)

In [32]:
model.fit(X,y,epochs=20,validation_split=0.2,batch_size=100,callbacks=[early_stop])

Epoch 1/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 104s 500ms/step - accuracy: 0.0690 - loss: 7.2727 - val_accuracy: 0.0561 - val_loss: 7.2352
Epoch 2/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 104s 516ms/step - accuracy: 0.0762 - loss: 6.6249 - val_accuracy: 0.0561 - val_loss: 7.3652
Epoch 3/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 105s 522ms/step - accuracy: 0.0741 - loss: 6.5873 - val_accuracy: 0.0561 - val_loss: 7.4652
Epoch 4/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 106s 526ms/step - accuracy: 0.0753 - loss: 6.5531 - val_accuracy: 0.0561 - val_loss: 7.5145


In [33]:
#to be continued